In [1]:
import sys
sys.path.append("../src/")

import numpy as np
from utils import flatten_dict


In [2]:
import wandb
import os
from hydra import (
    initialize, initialize_config_module, 
    initialize_config_dir, compose,
)
from hydra.utils import instantiate
from omegaconf import OmegaConf


if not OmegaConf.has_resolver("eval"):
    def resolver(x):
        import numpy as np
        return eval(x)
    OmegaConf.register_new_resolver("eval", resolver)



In [3]:
def simulate_rounds(model, rewards, actions_hist, X_global, y_global, batch_st, batch_end):
    np.random.seed(batch_st)
    actions_this_batch = model.predict(X_global[batch_st:batch_end, :]).astype('uint8')
    rewards.append(y_global[np.arange(batch_st, batch_end), actions_this_batch].sum())
    new_actions_hist = np.append(actions_hist, actions_this_batch)
    np.random.seed(batch_st)
    model.fit(
        X_global[:batch_end, :], 
        new_actions_hist, 
        y_global[np.arange(batch_end), new_actions_hist],
    )
    wandb.log(
        {
            "reward": y_global[np.arange(batch_st, batch_end), actions_this_batch].sum(),
            "regret": (
                y_global[np.arange(batch_st, batch_end),:].max(axis=1) 
                - 
                y_global[np.arange(batch_st, batch_end), actions_this_batch].flatten()
            ).sum(),
            "reward_avg": y_global[np.arange(batch_end), new_actions_hist].sum()/(batch_end/(batch_end-batch_st))
        }
    )
    return new_actions_hist


def run_model(model, X,y, cfg):
    batch_size = cfg.batch_size
    nchoices = y.shape[1]
    rewards = []
    first_batch = X[:batch_size, :]
    np.random.seed(cfg.seed)
    action_chosen = np.random.randint(nchoices, size=batch_size)
    rewards_received = y[np.arange(batch_size), action_chosen]
    actions = action_chosen.copy()
    model.fit(X=first_batch, a=action_chosen, r=rewards_received)
    for i in range(int(np.floor(X.shape[0] / batch_size))):
        batch_st = (i + 1) * batch_size
        batch_end = (i + 2) * batch_size
        batch_end = np.min([batch_end, X.shape[0]])
        if batch_end<=batch_st:
            continue
        actions = simulate_rounds(
            model,
            rewards,
            actions,
            X, 
            y,
            batch_st, 
            batch_end
        )
        wandb.log({"epoch": i})
        if cfg.debug and 3 < i: 
            break
    return rewards

In [4]:
def run_experiment(name):
    with initialize(
        version_base=None, 
        config_path="../conf/", 
    ):
        cfg = compose(
            config_name="config.yaml",
            # return_hydra_config=True, 
            overrides=[f"+experiment={name}"]
        )
    model = instantiate(cfg.model)
    parser = instantiate(cfg.parser)
    X,y = parser()
    flat_dict = flatten_dict(OmegaConf.to_container(cfg))
    run = wandb.init(project=cfg.project, config=flat_dict)
    reward = run_model(model, X,y, cfg)

In [5]:
for i in range(1,7):
    name = f"mediamill_{str(i).zfill(3)}"
    run_experiment(name)

wandb: Currently logged in as: roman-4erkasov-signup. Use `wandb login --relogin` to force relogin


/home/fatuus/rl-rairi/ve_cleanrl/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/fatuus/rl-rairi/ve_cleanrl/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mode

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
regret,▆▅▅▅▇▇▅█▆▆▅▇▆▄▆▅▆▆▁▄▃▅▄▆▇▇▄▅▅▇▅▆▆▅▇▅▃▃▃▆
reward,▂▄▃▄▂▃▄▁▄▁▅▂▄▃▄▆▄▄▅▄█▄▆▂▂▃▅▅▆▂▅▄▂▂▃▆▇▄█▂
reward_avg,▁▃▃▅▃▄▅▃▄▁▃▂▃▃▃▄▄▅▆▆▇▇█▇▇▇▇▇█▇█▇▇▆▆▆▇▇██
epoch,48
regret,93
reward,2
reward_avg,5.12


/home/fatuus/rl-rairi/ve_cleanrl/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/fatuus/rl-rairi/ve_cleanrl/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mode

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
regret,▆▄▅▅▇▇▅█▆▆▅▇▆▄▆▅▆▆▁▄▃▅▄▆▇▇▄▅▅▇▆▆▆▆▇▅▃▃▃▆
reward,▂▅▃▄▂▃▄▁▄▁▅▃▄▃▄▅▄▄▅▄█▄▆▂▂▃▅▅▆▂▅▄▂▂▃▆▇▄█▂
reward_avg,▁█▅▇▅▅▆▄▅▃▄▃▄▄▄▅▅▅▆▆████▇▇▇▇█▇█▇▇▆▆▆▇▇██
epoch,48
regret,93
reward,2
reward_avg,5.1


/home/fatuus/rl-rairi/ve_cleanrl/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/fatuus/rl-rairi/ve_cleanrl/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mode

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
regret,▆▄▅▅▇▇▅█▆▆▅▇▆▄▆▅▆▆▁▄▃▅▃▆▇▇▄▄▅▇▆▆▆▅▇▅▃▃▃▆
reward,▂▅▃▄▂▃▄▁▄▁▅▃▄▃▄▆▄▄▅▄█▄▆▂▂▃▅▅▆▂▅▄▂▂▃▆▇▄█▂
reward_avg,▁▇▅▆▄▅▆▄▄▂▄▃▄▄▄▅▅▅▆▆▇███▇▇▇▇█▇█▇▇▇▆▆▇▇██
epoch,48
regret,93
reward,2
reward_avg,5.22


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
regret,▇▆▅▅▇▇▆█▆▆▅▆▆▄▆▅▆▆▁▄▃▅▄▆▇▇▄▅▅▇▅▆▆▅▇▅▄▃▄▆
reward,▂▃▂▄▂▃▃▁▄▁▄▄▅▃▃▆▄▄▅▅█▄▅▁▂▃▅▄▅▂▅▄▃▂▃▆▅▄▇▂
reward_avg,▂▂▁▃▁▃▃▂▃▂▃▃▄▄▄▅▅▆▆▇███▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇██
epoch,48
regret,93
reward,2
reward_avg,4.82


/home/fatuus/rl-rairi/ve_cleanrl/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/fatuus/rl-rairi/ve_cleanrl/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_mode

InstantiationException: Error in call to target 'contextualbandits.online.EpsilonGreedy':
TypeError("__init__() got an unexpected keyword argument 'nsamples'")
full_key: model

In [8]:
name = f"mediamill_{str(6).zfill(3)}"
run_experiment(name)

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
regret,▇▆▅▅▇▆▅█▆▆▅▇▆▄▆▄▆▆▁▄▃▅▄▆▇▆▄▅▅▇▅▆▆▅▇▅▃▃▃▆
reward,▂▃▃▄▂▄▄▁▄▁▅▃▅▃▄▇▄▄▅▅█▄▅▂▂▄▅▅▅▂▆▄▂▂▃▆▇▄█▂
reward_avg,▁▁▁▂▂▃▄▃▄▂▃▃▄▄▄▅▅▆▆▇▇██▇▇▇▇▇█▇█▇▇▇▇▇▇▇██
epoch,48
regret,93
reward,2
reward_avg,5.24
